In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
pd.set_option('display.max_columns', 500) # show all columns
pd.options.display.max_rows = 200 # show 200 rows
        
import warnings
warnings.filterwarnings("ignore") #remove warning messages during csv import

In [26]:
dataPath = "car_features.csv"
raw_data_original = pd.read_csv(dataPath, nrows=110, index_col=0).transpose()

raw_data = raw_data_original.copy()

In [27]:
#### COMPANY NAMES

raw_data['Full Name'] = raw_data.index
raw_data['Company Name'] = raw_data['Full Name'].str.split(" ",expand=True)[1]

del raw_data['Full Name']

raw_data['Company Name'] = raw_data['Company Name'].str.replace("Alfa", "Alfa Romeo")
raw_data['Company Name'] = raw_data['Company Name'].str.replace("Aston", "Aston Martin")
raw_data['Company Name'] = raw_data['Company Name'].str.replace("Land", "Land Rover")
raw_data['Company Name'] = raw_data['Company Name'].str.replace("smart", "Smart")


# -------- replace na and tbd with np nan

raw_data.replace("NA", np.nan)
raw_data = raw_data.replace("- TBD –", 'NA')
raw_data = raw_data.replace("- TBD -", 'NA')
raw_data['EPA Fuel Economy Est - City (MPG)'] = raw_data['EPA Fuel Economy Est - City (MPG)'].str.replace(r"\(.*\)","")
raw_data = raw_data.replace("NA", np.nan)

# -------- cols with forbidden charac

raw_data = raw_data.rename(columns=lambda x: x.split(" (ft")[0])
raw_data['Passenger Volume'] = raw_data['Passenger Volume'].str.replace(r"\(.*\)","")

# -------- Clean MSRP and convert to float

raw_data.MSRP = raw_data.MSRP.str.replace("$", "")
raw_data.MSRP = raw_data.MSRP.str.replace(",", "")

# -------- Clean basic miles and convert to float

raw_data['Basic Miles/km'] = raw_data['Basic Miles/km'].str.replace(",", "")
raw_data['Basic Miles/km'] = raw_data['Basic Miles/km'].str.replace("Unlimited", "150000")
raw_data['Basic Miles/km'] = raw_data['Basic Miles/km'].str.replace("49999", "50000")

# -------- Clean Drivetrain Miles and convert to float

raw_data['Drivetrain Miles/km'] = raw_data['Drivetrain Miles/km'].str.replace(",", "")
raw_data['Drivetrain Miles/km'] = raw_data['Drivetrain Miles/km'].str.replace("Unlimited", "150000")

# -------- get Roadside Assistance Miles/km miles  as integer

raw_data['Roadside Assistance Miles/km'] = raw_data['Roadside Assistance Miles/km'].str.replace(",", "")
raw_data['Roadside Assistance Miles/km'] = raw_data['Roadside Assistance Miles/km'].str.replace("Unlimited", "100000")

# -------- get number of gears

raw_data['Transmission'] = raw_data['Transmission'].str.lower()
raw_data['Gears'] = raw_data['Transmission'].str.split("-speed", expand=True, n = 1)[0].str[-2:].str.strip()
raw_data.Gears = raw_data['Gears'].str.replace("le", "1")
raw_data.Gears = raw_data['Gears'].str.replace("ed", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("ic", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("es", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("er", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("ls", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("ve", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("to", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("de", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("ch", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("ct", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("rs", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("ft", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("al", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("s,", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("on", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("NA", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("co", "NA")
raw_data.Gears = raw_data['Gears'].str.replace("/8", "NA")


# -------- get max horsepower 
################ FIX THE FIRST SPLIT TO KEEP ONLY THE NUMBER

raw_data['Net Horsepower'] = raw_data['SAE Net Horsepower @ RPM'].str.split(" ",expand=True)[0].value_counts()
raw_data['Net Horsepower'] = raw_data['Net Horsepower'].astype(float)
raw_data.replace("NA", np.nan, inplace=True)


# -------- get max horsepower rpm 
############### FIX
raw_data['Net Horsepower RPM'] = raw_data['SAE Net Horsepower @ RPM'].str.split("@",expand=True)[1].str.strip()
raw_data['Net Horsepower RPM'] = raw_data['Net Horsepower RPM'].str.replace("- TBD -", "NA").str.replace("-TBD-", "NA")
raw_data['Net Horsepower RPM'] = raw_data['Net Horsepower RPM'].str[:4]

# -------- get max torque

raw_data['Net Torque'] = raw_data['SAE Net Torque @ RPM'].str.split(" ", expand=True)[0]
raw_data.replace("NA", np.nan, inplace=True)
raw_data['Net Torque'] = raw_data['Net Torque'].astype(float)

In [28]:
raw_data['Displacement (L)'] = raw_data['Displacement'].str.split("/", expand=True)[0].str[:3]
raw_data['Displacement (L)'] = raw_data['Displacement (L)'].str.replace('39.', '3.9')


# -------- displacement - cc

raw_data['Displacement (cc)'] = raw_data['Displacement'].str.split("/", expand=True)[1]
raw_data['Displacement (cc)'] = raw_data['Displacement (cc)'].str.replace('- TBD -', 'NA')
raw_data['Displacement (cc)'] = raw_data['Displacement (cc)'].str.replace('- TBD –', 'NA')
raw_data['Displacement (cc)'] = raw_data['Displacement (cc)'].str.replace('302 CID', 'NA')
raw_data['Displacement (cc)'] = raw_data['Displacement (cc)'].str.replace(' NA', 'NA')
# raw_data.loc['2018 Buick Envision Specs: AWD 4-Door Essence':'2018 Buick Envision Specs: AWD 4-Door Preferred',
# "Displacement (cc)"] = 'NA'

# -------- get rear tire width

raw_data["Rear Tire Width"] = raw_data["Rear Tire Size"].str.split("/").str.get(0).str[-3:].str.strip()
raw_data["Rear Tire Width"] = raw_data["Rear Tire Width"].replace('R20', 'NA').replace('18\"', 'NA').replace('R15', 'NA').replace('60A', 'NA').replace('R18', 'NA')
raw_data.replace("NA", np.nan, inplace=True)
raw_data["Rear Tire Width"] = raw_data["Rear Tire Width"].astype(float)

# -------- get front tire width

raw_data["Front Tire Width"] = raw_data["Front Tire Size"].str.split("/").str.get(0).str[-3:].str.strip()
#### FIX
raw_data["Front Tire Width"] = raw_data["Front Tire Width"].replace('R20', 'NA').replace('18\"', 'NA').replace('R15', 'NA').replace('60A', 'NA').replace('R18', 'NA')
raw_data.replace("NA", np.nan, inplace=True)
raw_data["Front Tire Width"] = raw_data["Front Tire Width"].astype(float)

# -------- get rear wheel size
#### FIX
raw_data["Rear Wheel Size"] = raw_data["Rear Wheel Size (in)"].str[:2].replace('P2', 'NA')
raw_data.replace("NA", np.nan, inplace=True)
raw_data["Rear Wheel Size"] = raw_data["Rear Wheel Size"].astype(float)
# -------- get front wheel size
#### FIX
raw_data["Front Wheel Size"] = raw_data["Front Wheel Size (in)"].str[:2].replace('P2', 'NA')
raw_data.replace("NA", np.nan, inplace=True)
raw_data["Front Wheel Size"]= raw_data["Front Wheel Size"].astype(float)
# -------- get tire rating

raw_data["Tire Rating"] = raw_data["Front Tire Size"].str.split("/").str.get(-1).str[-4]
raw_data["Tire Rating"] = raw_data["Tire Rating"].replace('5', 'NA')
raw_data["Tire Rating"] = raw_data["Tire Rating"].replace('0', 'NA')
raw_data["Tire Rating"] = raw_data["Tire Rating"].replace('1', 'NA')
raw_data["Tire Rating"] = raw_data["Tire Rating"].replace('2', 'NA')



# -------- get width ratio

raw_data["Tire Width Ratio"] = raw_data["Rear Tire Width"]/raw_data["Front Tire Width"]

# -------- get size ratio

raw_data["Wheel Size Ratio"] = raw_data["Rear Wheel Size"] / raw_data["Front Wheel Size"]

# -------- get tire ratio

raw_data["Tire Ratio"] = raw_data["Front Tire Size"].str.split("/").str.get(1).str[0]
raw_data["Tire Ratio"] = raw_data["Tire Ratio"].replace('Y', 'NA')

# -------- get year

raw_data["Year"] = raw_data.index.str[:4].astype(float)

In [29]:
# -------- edit drivetrain values

raw_data['Drivetrain'] = raw_data['Drivetrain'].str.replace('4-Wheel Drive', 'Four Wheel Drive')
raw_data['Drivetrain'] = raw_data['Drivetrain'].str.replace('Front wheel drive', 'Front Wheel Drive')
raw_data['Drivetrain'] = raw_data['Drivetrain'].str.replace('Four-Wheel Drive', 'Four Wheel Drive')

# -------- edit fuel system values

raw_data['Fuel System'] = raw_data['Fuel System'].str.replace('Turbocharged EFI', 'Electronic Fuel Injection')
raw_data['Fuel System'] = raw_data['Fuel System'].str.replace('Electric', 'Electronic Fuel Injection')
raw_data['Fuel System'] = raw_data['Fuel System'].str.replace('Sequential MPI (injection)', 'Sequential MPI')
raw_data['Fuel System'] = raw_data['Fuel System'].str.replace('SMPI', 'Sequential MPI')
raw_data['Fuel System'] = raw_data['Fuel System'].str.replace('EFI', 'Electronic Fuel Injection')
raw_data['Fuel System'] = raw_data['Fuel System'].str.replace('Direct Gasoline Injection', 'Direct Injection')

In [30]:
raw_data['Cylinders'] = raw_data['Engine Type'].str.split("-", expand=True)[1]
raw_data['Cylinders'] = raw_data['Cylinders'].str.replace("Cyl", "4")
raw_data['Cylinders'] = raw_data['Cylinders'].str.replace("in Electric I4", "4")


# -------- replace na by npnan

raw_data.replace("NA", np.nan, inplace=True)

# -------- convert all to float

raw_data.MSRP = raw_data.MSRP.astype(float)
raw_data["Tire Ratio"] = raw_data["Tire Ratio"].astype(float)
raw_data['Displacement (cc)'] = raw_data['Displacement (cc)'].astype(float)
raw_data['Displacement (L)'] = raw_data['Displacement (L)'].astype(float)

raw_data['Cylinders'] = raw_data['Cylinders'].str.replace('cyl', 'NA').str.replace('Pressure Turbo Gas I5', 'NA').str.replace('Turbocharged Gas V12', 'NA').str.replace('Scroll Turbocharged Gas I6', 'NA').str.replace('4 Turbocharged', 'NA').str.replace('Turbocharged Gas V8', 'NA')
raw_data.replace("NA", np.nan, inplace=True)
raw_data['Cylinders'] = raw_data['Cylinders'].astype(float)

raw_data['Net Horsepower RPM'] = raw_data['Net Horsepower RPM'].astype(float)

In [31]:
raw_data['Gears'] = raw_data['Gears'].astype(float)
raw_data['Roadside Assistance Miles/km'] = raw_data['Roadside Assistance Miles/km'].astype(float)
raw_data['Drivetrain Miles/km'] = raw_data['Drivetrain Miles/km'].astype(float)
raw_data['Basic Miles/km'] = raw_data['Basic Miles/km'].astype(float)

In [32]:
raw_data['Net Torque RPM'] = raw_data['SAE Net Torque @ RPM'].str.split().str.get(-1).str[-4:].str.strip()
raw_data['Net Torque RPM'] = raw_data['Net Torque RPM'].str.replace("- TBD -", "NA").str.replace('-', 'NA').str.replace('ined', 'NA').str.replace('E85', 'NA').str.replace('NA\)', 'NA').str.replace('est\)', 'NA')
raw_data.replace("NA", np.nan, inplace=True)
raw_data['Net Torque RPM'] = raw_data['Net Torque RPM'].astype(float)
raw_data['Net Torque RPM'] = raw_data['Net Torque RPM'].clip(lower=1000)



# -------- replace na by npnan

raw_data.replace("NA", np.nan, inplace=True)

# -------- convert all to float

raw_data.MSRP = raw_data.MSRP.astype(float)
raw_data["Tire Ratio"] = raw_data["Tire Ratio"].astype(float)
raw_data['Displacement (cc)'] = raw_data['Displacement (cc)'].astype(float)
raw_data['Displacement (L)'] = raw_data['Displacement (L)'].astype(float)
raw_data['Cylinders'] = raw_data['Cylinders'].astype(float)
raw_data['Net Horsepower RPM'] = raw_data['Net Horsepower RPM'].astype(float)
raw_data['Gears'] = raw_data['Gears'].astype(float)
raw_data['Roadside Assistance Miles/km'] = raw_data['Roadside Assistance Miles/km'].astype(float)
raw_data['Drivetrain Miles/km'] = raw_data['Drivetrain Miles/km'].astype(float)
raw_data['Basic Miles/km'] = raw_data['Basic Miles/km'].astype(float)

# -------- delete useless specs


specs_to_numeric = ['MSRP', 'Passenger Capacity', 'Passenger Doors',
                    'Base Curb Weight (lbs)', 'Second Shoulder Room (in)',
                    'Second Head Room (in)', 'Front Shoulder Room (in)',
                    'Second Hip Room (in)', 'Front Head Room (in)', 'Second Leg Room (in)', 'Front Hip Room (in)',
                    'Front Leg Room (in)', 'Width, Max w/o mirrors (in)', 'Track Width, Rear (in)',
                    'Height, Overall (in)', 'Wheelbase (in)', 'Track Width, Front (in)',
                    'Fuel Tank Capacity, Approx (gal)', 'EPA Fuel Economy Est - City (MPG)',
                    'EPA Fuel Economy Est - Hwy (MPG)',
                    'Fuel Economy Est-Combined (MPG)', 'Fourth Gear Ratio (:1)',
                    'Second Gear Ratio (:1)', 'Reverse Ratio (:1)', 'Fifth Gear Ratio (:1)',
                    'Third Gear Ratio (:1)', 'Final Drive Axle Ratio (:1)', 'First Gear Ratio (:1)',
                    'Sixth Gear Ratio (:1)', 'Passenger Volume',
                    'Front Brake Rotor Diam x Thickness (in)', 'Disc - Front (Yes or   )',
                    'Rear Brake Rotor Diam x Thickness (in)', 'Rear Wheel Size (in)',
                    'Rear Wheel Material', 'Spare Wheel Size (in)', 'Front Wheel Size (in)', 'Basic Miles/km',
                    'Basic Years', 'Corrosion Years', 'Drivetrain Miles/km', 'Drivetrain Years',
                    'Roadside Assistance Miles/km', 'Roadside Assistance Years', 'Year', 'Tire Ratio',
                    'Front Tire Width', 'Rear Tire Width', 'Displacement (cc)', 'Displacement (L)', 'Net Torque RPM',
                    'Net Torque', 'Gears', 'Net Horsepower', 'Net Horsepower RPM', 'Cylinders']

for i in specs_to_numeric:
    raw_data[i] = pd.to_numeric(raw_data[i], errors='coerce')
    
specs_to_delete = ['Gas Mileage', 'Engine', 'Engine Type', 'SAE Net Horsepower @ RPM', 'SAE Net Torque @ RPM',
                  'Displacement', 'Trans Description Cont.', 'Rear Tire Size', 'Front Tire Size', 'Rear Wheel Size (in)',
                  'Front Wheel Size (in)', 'Transmission', 'EPA Class', 'Brake ABS System', 'Disc - Front (Yes or   )',
                  'Brake Type', 'Disc - Rear (Yes or   )', 'Spare Tire Size', 'Spare Wheel Size (in)', 'Spare Wheel Material']
raw_data.drop(specs_to_delete, axis=1, inplace=True)

########### FIX TO 75% ########## -------- Identifying columns with NaN totalling more than 75% of elements

col_to_delete = raw_data.columns[raw_data.isna().sum() >= 0.75*len(raw_data)].tolist()

raw_data.drop(col_to_delete, axis=1, inplace=True)


#
raw_data.index.name = 'carName'
raw_data.reset_index(inplace=True)
raw_data.head()

,carName,MSRP,Style Name,Drivetrain,Passenger Capacity,Passenger Doors,Body Style,EPA Classification,Base Curb Weight (lbs),Front Hip Room (in),Front Leg Room (in),Second Shoulder Room (in),Passenger Volume,Second Head Room (in),Front Shoulder Room (in),Second Hip Room (in),Front Head Room (in),Second Leg Room (in),Wheelbase (in),Min Ground Clearance (in),"Track Width, Front (in)","Width, Max w/o mirrors (in)","Track Width, Rear (in)","Height, Overall (in)",Cargo Volume to Seat 1,Cargo Volume to Seat 2,"Fuel Tank Capacity, Approx (gal)",Fuel Economy Est-Combined (MPG),EPA Fuel Economy Est - City (MPG),EPA Fuel Economy Est - Hwy (MPG),Engine Order Code,Fuel System,First Gear Ratio (:1),Sixth Gear Ratio (:1),Fourth Gear Ratio (:1),Trans Order Code,Second Gear Ratio (:1),Reverse Ratio (:1),Fifth Gear Ratio (:1),Trans Type,Third Gear Ratio (:1),Final Drive Axle Ratio (:1),Rear Brake Rotor Diam x Thickness (in),Front Brake Rotor Diam x Thickness (in),Steering Type,Turning Diameter - Curb to Curb,Front Tire Order Code,Rear Tire Order Code,Front Wheel Material,Suspension Type - Front,Suspension Type - Rear (Cont.),Suspension Type - Rear,Suspension Type - Front (Cont.),Air Bag-Frontal-Driver,Air Bag-Frontal-Passenger,Air Bag-Passenger Switch (On/Off),Air Bag-Side Body-Front,Air Bag-Side Body-Rear,Air Bag-Side Head-Front,Air Bag-Side Head-Rear,Brakes-ABS,Child Safety Rear Door Locks,Daytime Running Lights,Traction Control,Night Vision,Rollover Protection Bars,Fog Lamps,Parking Aid,Tire Pressure Monitor,Back-Up Camera,Stability Control,Other Features,Basic Miles/km,Basic Years,Corrosion Miles/km,Corrosion Years,Drivetrain Miles/km,Drivetrain Years,Roadside Assistance Miles/km,Roadside Assistance Years,Maximum Alternator Capacity (amps),Cold Cranking Amps @ 0° F (Primary),Wt Distributing Hitch - Max Tongue Wt. (lbs),Company Name,Gears,Net Horsepower RPM,Net Torque,Displacement (L),Displacement (cc),Rear Tire Width,Front Tire Width,Rear Wheel Size,Front Wheel Size,Tire Rating,Tire Width Ratio,Wheel Size Ratio,Tire Ratio,Year,Cylinders,Net Torque RPM
0,2019 Acura RDX Specs: FWD w/Technology Pkg,40600.0,FWD w/Technology Pkg,Front Wheel Drive,5,4,Sport Utility,Small Sport Utility Vehicles 2WD,3790.0,55.0,41.6,56.6,104.0,38.3,59.7,49.9,39.6,38.4,108.3,5.7,64.2,74.8,64.7,65.7,58.9,29.5,17.1,24.0,22.0,28.0,NaN,Gasoline Direct Injection,5.25,1.0,1.6,NaN,3.27,3.97,1.3,10,2.19,4.17,12.2,12.4,Rack-Pinion,39,NaN,NaN,Aluminum,Strut,Multi-Link,Multi-Link,Strut,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,Yes,Yes,Yes,Vehicle Stability Assist Electronic Stability ...,50000.0,4.0,Unlimited,5.0,70000.0,6.0,50000.0,4.0,NaN,NaN,NaN,Acura,10.0,6500.0,280.0,2.0,122.0,235.0,235.0,19.0,19.0,H,1.0,1.0,5.0,2019.0,4.0,1600.0
1,2019 Acura RDX Specs: FWD w/Advance Pkg,45500.0,FWD w/Advance Pkg,Front Wheel Drive,5,4,Sport Utility,Small Sport Utility Vehicles 2WD,3829.0,55.0,41.6,56.6,104.0,38.3,59.7,49.9,39.6,38.4,108.3,5.7,64.2,74.8,64.7,65.7,58.9,29.5,17.1,24.0,22.0,28.0,NaN,Gasoline Direct Injection,5.25,1.0,1.6,NaN,3.27,3.97,1.3,10,2.19,4.17,12.2,12.4,Rack-Pinion,39,NaN,NaN,Aluminum,Strut,Multi-Link,Multi-Link,Strut,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,Yes,Vehicle Stability Assist Electronic Stability ...,50000.0,4.0,Unlimited,5.0,70000.0,6.0,50000.0,4.0,NaN,NaN,NaN,Acura,10.0,6500.0,280.0,2.0,122.0,235.0,235.0,19.0,19.0,H,1.0,1.0,5.0,2019.0,4.0,1600.0
2,2019 Acura RDX Specs: FWD w/A-Spec Pkg,43600.0,FWD w/A-Spec Pkg,Front Wheel Drive,5,4,Sport Utility,Small Sport Utility Vehicles 2WD,3821.0,55.0,41.6,56.6,104.0,38.3,59.7,49.9,39.6,38.4,108.3,5.7,64.2,74.8,64.7,65.7,58.9,29.5,17.1,24.0,22.0,27.0,NaN,Gasoline Direct Injection,5.25,1.0,1.6,NaN,3.27,3.97,1.3,10,2.19,4.17,12.2,12.4,Rack-Pinion,39,NaN,NaN,Aluminum,Strut,Multi-Link,Multi-Link,Strut,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,Yes,Vehicle Stability Assist Electronic Stability ...,50000.0,4.0,Unlimited,5.0,70000.0,6.0,50000.0,4.0,NaN,NaN,NaN,Acura,1